In [1]:
import numpy as np 
import pandas as pd 
import xarray as xr 
import cartopy 
import glob
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
import scipy 
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline, make_union
import warnings
import seaborn as sns
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import matplotlib.offsetbox as offsetbox
import seaborn.objects



In [2]:
root_path='/data/keeling/a/rytam2/ccf_model_spread/data/preprocessed/'
obs_kernel_path='/data/keeling/a/rytam2/a/kernels/obs/'
kernel_path='/data/keeling/a/rytam2/a/kernels/gcms/'

In [3]:
#AMIP data
ts_amip = xr.open_mfdataset(root_path+'ts_amip_CMIP5&6_198201_200812.nc')
eis_amip = xr.open_mfdataset(root_path+'eis_amip_CMIP5&6_198201_200812.nc')
tadv_amip = xr.open_mfdataset(root_path+'tadv_amip_CMIP5&6_198201_200812.nc')*24*3600
rh_amip = xr.open_mfdataset(root_path+'hur_amip_CMIP5&6_198201_200812.nc')
omega_amip = xr.open_mfdataset(root_path+'wap_amip_CMIP5&6_198201_200812.nc')*864
ws_amip = xr.open_mfdataset(root_path+'ws_amip_CMIP5&6_198201_200812.nc')

#historical data
ts_hist = xr.open_mfdataset(root_path+'ts_obshist_CMIP5&6_2008.nc')
eis_hist = xr.open_mfdataset(root_path+'eis_obshist_CMIP5&6_2008.nc')
tadv_hist = xr.open_mfdataset(root_path+'tadv_obshist_CMIP5&6_2008.nc')*24*3600
rh_hist = xr.open_mfdataset(root_path+'hur_obshist_CMIP5&6_2008.nc')
omega_hist = xr.open_mfdataset(root_path+'wap_obshist_CMIP5&6_2008.nc')*864
ws_hist = xr.open_mfdataset(root_path+'ws_obshist_CMIP5&6_2008.nc')

#abrupt data
ts_abrupt = xr.open_mfdataset(root_path+'ts_4xCO2_CMIP5&6_Y1-150.nc') #SST 
eis_abrupt = xr.open_mfdataset(root_path+'eis_4xCO2_CMIP5&6_Y1-150.nc') #EIS 
tadv_abrupt = xr.open_mfdataset(root_path+'tadv_4xCO2_CMIP5&6_Y1-150.nc')*24*3600 #Temperature Advection 
rh_abrupt = xr.open_mfdataset(root_path+'hur_4xCO2_CMIP5&6_Y1-150.nc') # Relative Humidity 
omega_abrupt = xr.open_mfdataset(root_path+'wap_4xCO2_CMIP5&6_Y1-150.nc')*864 # Vertical advection 
ws_abrupt = xr.open_mfdataset(root_path+'ws_4xCO2_updated_CMIP5&6_Y1-150.nc') #Surface Windspeed 

# Rename Model Names 
name_list = ['CCSM4','CanESM2','CanESM5','E3SM-1-0','GFDL-CM4','HadGEM2','HadGEM3-GC31-LL','IPSL-CM6A-LR','MIROC-ES2L','MIROC-ESM',
                     'MIROC5','MIROC6','MPI-ESM','MRI-CGCM3','MRI-ESM2-0','UKESM1-0-LL']

model_list=['ccsm4','canam4','canesm5','e3sm','gfdl','hg2','hg3','ipsl','mies2l','miesm','mi5','mi6','mpi','mrcgcm','mresm','ukesm']

rename_dict = dict(zip(sorted(name_list), sorted(model_list)))

ts_abrupt = ts_abrupt.rename(rename_dict) 
eis_abrupt = eis_abrupt.rename(rename_dict)
tadv_abrupt = tadv_abrupt.rename(rename_dict)
rh_abrupt = rh_abrupt.rename(rename_dict)
omega_abrupt = omega_abrupt.rename(rename_dict)
ws_abrupt = ws_abrupt.rename(rename_dict)

In [4]:
# Load Kernels 
kernels_list = glob.glob(kernel_path+'*.nc');

dict_sst_kernels = {}
dict_eis_kernels = {}
dict_tadv_kernels = {}
dict_rh_kernels = {}
dict_omega_kernels = {}
dict_ws_kernels = {}

modelname_list=['ccsm4','canam4','canesm5','e3sm','gfdl','hg2','hg3','ipsl',
                'mies2l','miesm','mi5','mi6','mpi','mrcgcm','mresm','ukesm'] #order must be sticktly followed to match kernels_list


# Get dRdxi from files
for filename in kernels_list:
    kernel=xr.open_mfdataset(filename)
    j = kernels_list.index(filename)
    modelname=modelname_list[j]
    dict_sst_kernels["%s"%modelname] = kernel.isel(i=0).dRdxi
    dict_eis_kernels["%s"%modelname] = kernel.isel(i=1).dRdxi
    dict_tadv_kernels["%s"%modelname] = kernel.isel(i=2).dRdxi
    dict_rh_kernels["%s"%modelname] = kernel.isel(i=3).dRdxi
    dict_omega_kernels["%s"%modelname] = kernel.isel(i=4).dRdxi
    dict_ws_kernels["%s"%modelname] = kernel.isel(i=5).dRdxi
    
# Define new coords 
coords = {'lat':(['latitude'],kernel.lat.values),
         'lon':(['longitude'],kernel.lon.values)}

# Create kernel xarray variables 
sst_kernels = xr.Dataset(data_vars=dict_sst_kernels,coords=coords);
eis_kernels = xr.Dataset(data_vars=dict_eis_kernels,coords=coords);
tadv_kernels = xr.Dataset(data_vars=dict_tadv_kernels,coords=coords);
rh_kernels = xr.Dataset(data_vars=dict_rh_kernels,coords=coords);
omega_kernels = xr.Dataset(data_vars=dict_omega_kernels,coords=coords);
ws_kernels = xr.Dataset(data_vars=dict_ws_kernels,coords=coords);

# Rename to avoid duplicate dimensions
sst_kernels = sst_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'});
eis_kernels = eis_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'});
tadv_kernels = tadv_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'});
rh_kernels = rh_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'});
omega_kernels = omega_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'});
ws_kernels = ws_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'});

In [5]:
obskernels_list = glob.glob(obs_kernel_path+'*.nc');

for filename in obskernels_list:
    kernel=xr.open_mfdataset(filename)

dict_sst_kernels = {}
dict_eis_kernels = {}
dict_tadv_kernels = {}
dict_rh_kernels = {}
dict_omega_kernels = {}
dict_ws_kernels = {}

obskernel_namelist = ['ceres','isccp','modis','patmosx']

modelname_list=['ccsm4','canam4','canesm5','e3sm','gfdl','hg2','hg3','ipsl',
                'mies2l','miesm','mi5','mi6','mpi','mrcgcm','mresm','ukesm'] #order must be sticktly followed to match kernels_list



# Get dRdxi from files
for filename in obskernels_list:
    kernel=xr.open_mfdataset(filename)
    j = obskernels_list.index(filename)
    obsname=obskernel_namelist[j]
    dict_sst_kernels["%s"%obsname] = kernel.isel(i=0).dRdxi
    dict_eis_kernels["%s"%obsname] = kernel.isel(i=1).dRdxi
    dict_tadv_kernels["%s"%obsname] = kernel.isel(i=2).dRdxi
    dict_rh_kernels["%s"%obsname] = kernel.isel(i=3).dRdxi
    dict_omega_kernels["%s"%obsname] = kernel.isel(i=4).dRdxi
    dict_ws_kernels["%s"%obsname] = kernel.isel(i=5).dRdxi
    
# Define new coords 
coords = {'lat':(['latitude'],kernel.latitude.values),
         'lon':(['longitude'],kernel.longitude.values)}

# Create kernel xarray variables 
sst_obs_kernels = xr.Dataset(data_vars=dict_sst_kernels,coords=coords);
eis_obs_kernels = xr.Dataset(data_vars=dict_eis_kernels,coords=coords);
tadv_obs_kernels = xr.Dataset(data_vars=dict_tadv_kernels,coords=coords);
rh_obs_kernels = xr.Dataset(data_vars=dict_rh_kernels,coords=coords);
omega_obs_kernels = xr.Dataset(data_vars=dict_omega_kernels,coords=coords);
ws_obs_kernels = xr.Dataset(data_vars=dict_ws_kernels,coords=coords);

# Rename to avoid duplicate dimensions
sst_obs_kernels = sst_obs_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'});
eis_obs_kernels = eis_obs_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'});
tadv_obs_kernels = tadv_obs_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'});
rh_obs_kernels = rh_obs_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'});
omega_obs_kernels = omega_obs_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'});
ws_obs_kernels = ws_obs_kernels.swap_dims(dims_dict={'latitude':'lat','longitude':'lon'});

In [6]:
# Remove seasonal cycle from measurements 
def remove_seasonality(x):
    return x - x.mean(dim='time')

#AMIP
# Group data by month and remove seasonality
ts_noseason_amip = ts_amip.groupby('time.month').apply(remove_seasonality)#.assign_coords({'time': np.arange(1850.0,2000,1/12)}) #set decimal year 
eis_noseason_amip = eis_amip.groupby('time.month').apply(remove_seasonality)#.assign_coords({'time': np.arange(1850.0,2000,1/12)}) #set decimal year
tadv_noseason_amip = tadv_amip.groupby('time.month').apply(remove_seasonality)#.assign_coords({'time': np.arange(1850.0,2000,1/12)}) #set decimal year 
rh_noseason_amip = rh_amip.groupby('time.month').apply(remove_seasonality)#.assign_coords({'time': np.arange(1850.0,2000,1/12)}) #set decimal year 
omega_noseason_amip = omega_amip.groupby('time.month').apply(remove_seasonality)#.assign_coords({'time': np.arange(1850.0,2000,1/12)}) #set decimal year 
ws_noseason_amip = ws_amip.groupby('time.month').apply(remove_seasonality)#.assign_coords({'time': np.arange(1850.0,2000,1/12)}) #set decimal year 


#Hist
# Group data by month and remove seasonality
ts_noseason_hist = ts_hist.groupby('time.month').apply(remove_seasonality)#.assign_coords({'time': np.arange(1850.0,2000,1/12)}) #set decimal year 
eis_noseason_hist = eis_hist.groupby('time.month').apply(remove_seasonality)#.assign_coords({'time': np.arange(1850.0,2000,1/12)}) #set decimal year
tadv_noseason_hist = tadv_hist.groupby('time.month').apply(remove_seasonality)#.assign_coords({'time': np.arange(1850.0,2000,1/12)}) #set decimal year 
rh_noseason_hist = rh_hist.groupby('time.month').apply(remove_seasonality)#.assign_coords({'time': np.arange(1850.0,2000,1/12)}) #set decimal year 
omega_noseason_hist = omega_hist.groupby('time.month').apply(remove_seasonality)#.assign_coords({'time': np.arange(1850.0,2000,1/12)}) #set decimal year 
ws_noseason_hist = ws_hist.groupby('time.month').apply(remove_seasonality)#.assign_coords({'time': np.arange(1850.0,2000,1/12)}) #set decimal year 


#Abrupt
# Group data by month and remove seasonality
ts_noseason_abrupt = ts_abrupt.groupby('time.month').apply(remove_seasonality)#.assign_coords({'time': np.arange(1850.0,2000,1/12)}) #set decimal year 
eis_noseason_abrupt = eis_abrupt.groupby('time.month').apply(remove_seasonality)#.assign_coords({'time': np.arange(1850.0,2000,1/12)}) #set decimal year
tadv_noseason_abrupt = tadv_abrupt.groupby('time.month').apply(remove_seasonality)#.assign_coords({'time': np.arange(1850.0,2000,1/12)}) #set decimal year 
rh_noseason_abrupt = rh_abrupt.groupby('time.month').apply(remove_seasonality)#.assign_coords({'time': np.arange(1850.0,2000,1/12)}) #set decimal year 
omega_noseason_abrupt = omega_abrupt.groupby('time.month').apply(remove_seasonality)#.assign_coords({'time': np.arange(1850.0,2000,1/12)}) #set decimal year 
ws_noseason_abrupt = ws_abrupt.groupby('time.month').apply(remove_seasonality)#.assign_coords({'time': np.arange(1850.0,2000,1/12)}) #set decimal year 

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 27 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 27 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 27 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 27 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 27 times mo

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 27 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 27 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 27 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 27 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 27 times mo

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 27 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 27 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 27 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 27 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 27 times mo

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 27 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 27 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 27 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 27 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 27 times mo

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 150 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 150 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 150 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 150 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 150 tim

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 150 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 150 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 150 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 150 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 150 tim

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 150 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 150 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 150 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 150 times more chunks
  return self.array[key]
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing with an out-of-order index is generating 150 tim

In [7]:
# Remove seasonal cycle from measurements 
def approx_change(x,degree):
    if degree == 1: 
        poly=x.polyfit(dim='time', deg=1)
        fit = xr.polyval(x['time'],poly.polyfit_coefficients)
        change = fit.isel(time=-1)-fit.isel(time=0)
        return change
    
def normalize(x): 
    glob = x.mean('lat').mean('lon')
    norm = x/glob
    return x

def ensemble_mean(x):
    return x.to_array(dim='models').mean('models')

def make_textbox(axes, string):

    box1 = offsetbox.TextArea(string,textprops=dict(fontsize=12,ha='left',fontweight='bold'))
    anchored_box = offsetbox.AnchoredOffsetbox(loc=3,
                                 child=box1, pad=0.2,
                                 frameon=False,
                                 bbox_to_anchor=(0,1),
                                 bbox_transform=axes.transAxes,
                                 borderpad=.2)
    axes.add_artist(anchored_box)
    
    return


def shrink_width_fig1(axes):
    box = axes.get_position()
    box.x0 = box.x0 + .015
    box.x1 = box.x1 + .008
    axes.set_position(box)
    return None

def shrink_width_suppfig1(axes):
    box = axes.get_position()
    box.x0 = box.x0 + 0.098
    box.x1 = box.x1 - 0.062
    axes.set_position(box)
    return None

def shrink_width_fig2(axes):
    box = axes.get_position()
    box.x0 = box.x0 + .0125
    box.x1 = box.x1 - 0.0125
    axes.set_position(box)
    return None

def shrink_width_suppfig2(axes):
    box = axes.get_position()
    box.x0 = box.x0 + 0.11
    box.x1 = box.x1 - 0.11
    axes.set_position(box)
    return None

# Get Changes in CCF (per temperature change)

In [8]:
# Get approximated change in CCF across period 

#AMIP
da_list=[approx_change(ts_noseason_amip[var],1).rename(var) for var in modelname_list]
dSST_hat_amip = xr.merge(da_list)
dT_glob_amip = ensemble_mean(dSST_hat_amip.mean('lat').mean('lon')).values
dSST_hat_amip = dSST_hat_amip/dT_glob_amip
da_list=[approx_change(eis_noseason_amip[var],1).rename(var) for var in modelname_list]
dEIS_hat_amip = xr.merge(da_list)/dT_glob_amip
da_list=[approx_change(tadv_noseason_amip[var],1).rename(var) for var in modelname_list]
dTadv_hat_amip = xr.merge(da_list)/dT_glob_amip
da_list=[approx_change(rh_noseason_amip[var],1).rename(var) for var in modelname_list]
dRH_hat_amip = xr.merge(da_list)/dT_glob_amip
da_list=[approx_change(omega_noseason_amip[var],1).rename(var) for var in modelname_list]
dOmega_hat_amip = xr.merge(da_list)/dT_glob_amip
da_list=[approx_change(ws_noseason_amip[var],1).rename(var) for var in modelname_list]
dWS_hat_amip = xr.merge(da_list)/dT_glob_amip

# Historical 
da_list=[approx_change(ts_noseason_hist[var],1).rename(var) for var in modelname_list]
dSST_hat_hist = xr.merge(da_list)
dT_glob_hist = ensemble_mean(dSST_hat_hist.mean('lat').mean('lon'))
dSST_hat_hist = dSST_hat_hist/dT_glob_hist
da_list=[approx_change(eis_noseason_hist[var],1).rename(var) for var in modelname_list]
dEIS_hat_hist = xr.merge(da_list)/dT_glob_hist
da_list=[approx_change(tadv_noseason_hist[var],1).rename(var) for var in modelname_list]
dTadv_hat_hist = xr.merge(da_list)/dT_glob_hist
da_list=[approx_change(rh_noseason_hist[var],1).rename(var) for var in modelname_list]
dRH_hat_hist = xr.merge(da_list)/dT_glob_hist
da_list=[approx_change(omega_noseason_hist[var],1).rename(var) for var in modelname_list]
dOmega_hat_hist = xr.merge(da_list)/dT_glob_hist
da_list=[approx_change(ws_noseason_hist[var],1).rename(var) for var in modelname_list]
dWS_hat_hist = xr.merge(da_list)/dT_glob_hist

# Abrupt Fast response
da_list=[approx_change(ts_noseason_abrupt[var].isel(time=slice(0,239)),1).rename(var) for var in modelname_list]
dSST_hat_fast = xr.merge(da_list)
dT_glob_fast = ensemble_mean(dSST_hat_fast.mean('lat').mean('lon'))
dSST_hat_fast = dSST_hat_fast/dT_glob_fast
da_list=[approx_change(eis_noseason_abrupt[var].isel(time=slice(0,239)),1).rename(var) for var in modelname_list]
dEIS_hat_fast = xr.merge(da_list)/dT_glob_fast
da_list=[approx_change(tadv_noseason_abrupt[var].isel(time=slice(0,239)),1).rename(var) for var in modelname_list]
dTadv_hat_fast = xr.merge(da_list)/dT_glob_fast
da_list=[approx_change(rh_noseason_abrupt[var].isel(time=slice(0,239)),1).rename(var) for var in modelname_list]
dRH_hat_fast = xr.merge(da_list)/dT_glob_fast
da_list=[approx_change(omega_noseason_abrupt[var].isel(time=slice(0,239)),1).rename(var) for var in modelname_list]
dOmega_hat_fast = xr.merge(da_list)/dT_glob_fast
da_list=[approx_change(ws_noseason_abrupt[var].isel(time=slice(0,239)),1).rename(var) for var in modelname_list]
dWS_hat_fast = xr.merge(da_list)/dT_glob_fast

# Abrupt Slow response
da_list=[approx_change(ts_noseason_abrupt[var].isel(time=slice(240,1800)),1).rename(var) for var in modelname_list]
dSST_hat_slow = xr.merge(da_list)
dT_glob_slow = ensemble_mean(dSST_hat_slow.mean('lat').mean('lon'))
dSST_hat_slow = dSST_hat_slow/dT_glob_slow
da_list=[approx_change(eis_noseason_abrupt[var].isel(time=slice(240,1800)),1).rename(var) for var in modelname_list]
dEIS_hat_slow = xr.merge(da_list)/dT_glob_slow
da_list=[approx_change(tadv_noseason_abrupt[var].isel(time=slice(240,1800)),1).rename(var) for var in modelname_list]
dTadv_hat_slow = xr.merge(da_list)/dT_glob_slow
da_list=[approx_change(rh_noseason_abrupt[var].isel(time=slice(240,1800)),1).rename(var) for var in modelname_list]
dRH_hat_slow = xr.merge(da_list)/dT_glob_slow
da_list=[approx_change(omega_noseason_abrupt[var].isel(time=slice(240,1800)),1).rename(var) for var in modelname_list]
dOmega_hat_slow = xr.merge(da_list)/dT_glob_slow
da_list=[approx_change(ws_noseason_abrupt[var].isel(time=slice(240,1800)),1).rename(var) for var in modelname_list]
dWS_hat_slow = xr.merge(da_list)/dT_glob_slow

In [17]:
# Ensemble Mean Changes

# AMIP
dSST_hat_bar_amip = ensemble_mean(dSST_hat_amip)
dEIS_hat_bar_amip = ensemble_mean(dEIS_hat_amip)
dTadv_hat_bar_amip = ensemble_mean(dTadv_hat_amip)
dRH_hat_bar_amip = ensemble_mean(dRH_hat_amip)
dOmega_hat_bar_amip = ensemble_mean(dOmega_hat_amip)
dWS_hat_bar_amip = ensemble_mean(dWS_hat_amip)

# Hist
dSST_hat_bar_hist = ensemble_mean(dSST_hat_hist)
dEIS_hat_bar_hist = ensemble_mean(dEIS_hat_hist)
dTadv_hat_bar_hist = ensemble_mean(dTadv_hat_hist)
dRH_hat_bar_hist = ensemble_mean(dRH_hat_hist)
dOmega_hat_bar_hist = ensemble_mean(dOmega_hat_hist)
dWS_hat_bar_hist = ensemble_mean(dWS_hat_hist)

# Abrupt Fast response
dSST_hat_bar_fast = ensemble_mean(dSST_hat_fast)
dEIS_hat_bar_fast = ensemble_mean(dEIS_hat_fast)
dTadv_hat_bar_fast = ensemble_mean(dTadv_hat_fast)
dRH_hat_bar_fast = ensemble_mean(dRH_hat_fast)
dOmega_hat_bar_fast = ensemble_mean(dOmega_hat_fast)
dWS_hat_bar_fast = ensemble_mean(dWS_hat_fast)

# Abrupt Slow response
dSST_hat_bar_slow = ensemble_mean(dSST_hat_slow)
dEIS_hat_bar_slow = ensemble_mean(dEIS_hat_slow)
dTadv_hat_bar_slow = ensemble_mean(dTadv_hat_slow)
dRH_hat_bar_slow = ensemble_mean(dRH_hat_slow)
dOmega_hat_bar_slow = ensemble_mean(dOmega_hat_slow)
dWS_hat_bar_slow = ensemble_mean(dWS_hat_slow)


# Kernels 
sst_kernels_bar = ensemble_mean(sst_kernels)
eis_kernels_bar = ensemble_mean(eis_kernels)
tadv_kernels_bar = ensemble_mean(tadv_kernels)
rh_kernels_bar = ensemble_mean(rh_kernels)
omega_kernels_bar = ensemble_mean(omega_kernels)
ws_kernels_bar = ensemble_mean(ws_kernels)

# Get Feedback Changes 

#### Kernels * dCCF

In [10]:
# AMIP
dR_SST_hat_amip = dSST_hat_amip * sst_kernels
dR_EIS_hat_amip = dEIS_hat_amip * eis_kernels
dR_Tadv_hat_amip = dTadv_hat_amip * tadv_kernels
dR_RH_hat_amip = dRH_hat_amip * rh_kernels
dR_Omega_hat_amip = dOmega_hat_amip * omega_kernels
dR_WS_hat_amip = dWS_hat_amip * ws_kernels
# Ensemble Mean of dCCF_hat
dR_SST_hat_amip_bar = ensemble_mean(dR_SST_hat_amip)
dR_EIS_hat_amip_bar = ensemble_mean(dR_EIS_hat_amip)
dR_Tadv_hat_amip_bar = ensemble_mean(dR_Tadv_hat_amip)
dR_RH_hat_amip_bar = ensemble_mean(dR_RH_hat_amip)
dR_Omega_hat_amip_bar = ensemble_mean(dR_Omega_hat_amip)
dR_WS_hat_amip_bar = ensemble_mean(dR_WS_hat_amip)


# Historical
dR_SST_hat_hist = dSST_hat_hist * sst_kernels
dR_EIS_hat_hist = dEIS_hat_hist * eis_kernels
dR_Tadv_hat_hist = dTadv_hat_hist * tadv_kernels
dR_RH_hat_hist = dRH_hat_hist * rh_kernels
dR_Omega_hat_hist = dOmega_hat_hist * omega_kernels
dR_WS_hat_hist = dWS_hat_hist * ws_kernels
# Ensemble Mean of dCCF_hat
dR_SST_hat_hist_bar = ensemble_mean(dR_SST_hat_hist)
dR_EIS_hat_hist_bar= ensemble_mean(dR_EIS_hat_hist)
dR_Tadv_hat_hist_bar = ensemble_mean(dR_Tadv_hat_hist)
dR_RH_hat_hist_bar = ensemble_mean(dR_RH_hat_hist)
dR_Omega_hat_hist_bar = ensemble_mean(dR_Omega_hat_hist)
dR_WS_hat_hist_bar = ensemble_mean(dR_WS_hat_hist)


# Abrupt fast response
dR_SST_hat_fast = dSST_hat_fast * sst_kernels
dR_EIS_hat_fast = dEIS_hat_fast * eis_kernels
dR_Tadv_hat_fast = dTadv_hat_fast * tadv_kernels
dR_RH_hat_fast = dRH_hat_fast * rh_kernels
dR_Omega_hat_fast = dOmega_hat_fast * omega_kernels
dR_WS_hat_fast = dWS_hat_fast * ws_kernels
# Ensemble Mean of dCCF_hat
dR_SST_hat_fast_bar = ensemble_mean(dR_SST_hat_fast)
dR_EIS_hat_fast_bar = ensemble_mean(dR_EIS_hat_fast)
dR_Tadv_hat_fast_bar = ensemble_mean(dR_Tadv_hat_fast)
dR_RH_hat_fast_bar = ensemble_mean(dR_RH_hat_fast)
dR_Omega_hat_fast_bar = ensemble_mean(dR_Omega_hat_fast)
dR_WS_hat_fast_bar = ensemble_mean(dR_WS_hat_fast)


# Abrupt slow response
dR_SST_hat_slow = dSST_hat_slow * sst_kernels
dR_EIS_hat_slow = dEIS_hat_slow * eis_kernels
dR_Tadv_hat_slow = dTadv_hat_slow * tadv_kernels
dR_RH_hat_slow = dRH_hat_slow * rh_kernels
dR_Omega_hat_slow = dOmega_hat_slow * omega_kernels
dR_WS_hat_slow = dWS_hat_slow * ws_kernels
# Ensemble Mean of dCCF_hat
dR_SST_hat_slow_bar = ensemble_mean(dR_SST_hat_slow)
dR_EIS_hat_slow_bar = ensemble_mean(dR_EIS_hat_slow)
dR_Tadv_hat_slow_bar = ensemble_mean(dR_Tadv_hat_slow)
dR_RH_hat_slow_bar = ensemble_mean(dR_RH_hat_slow)
dR_Omega_hat_slow_bar = ensemble_mean(dR_Omega_hat_slow)
dR_WS_hat_slow_bar = ensemble_mean(dR_WS_hat_slow)


#Total low-cloud feedback
dR_hat_amip = dR_SST_hat_amip + dR_EIS_hat_amip + dR_Tadv_hat_amip + dR_RH_hat_amip + dR_Omega_hat_amip + dR_WS_hat_amip
dR_hat_fast = dR_SST_hat_fast + dR_EIS_hat_fast + dR_Tadv_hat_fast + dR_RH_hat_fast + dR_Omega_hat_fast + dR_WS_hat_fast
dR_hat_slow = dR_SST_hat_slow + dR_EIS_hat_slow + dR_Tadv_hat_slow + dR_RH_hat_slow + dR_Omega_hat_slow + dR_WS_hat_slow
dR_hat_hist = dR_SST_hat_hist + dR_EIS_hat_hist + dR_Tadv_hat_hist + dR_RH_hat_hist + dR_Omega_hat_hist + dR_WS_hat_hist

#### CERES

In [11]:
# Obs AMIP
dR_SST_hat_amip_ceres = dSST_hat_amip * sst_obs_kernels.ceres
dR_EIS_hat_amip_ceres = dEIS_hat_amip * eis_obs_kernels.ceres
dR_Tadv_hat_amip_ceres = dTadv_hat_amip * tadv_obs_kernels.ceres
dR_RH_hat_amip_ceres = dRH_hat_amip * rh_obs_kernels.ceres
dR_Omega_hat_amip_ceres = dOmega_hat_amip * omega_obs_kernels.ceres
dR_WS_hat_amip_ceres = dWS_hat_amip * ws_obs_kernels.ceres

# Historical
dR_SST_hat_hist_ceres = dSST_hat_hist * sst_obs_kernels.ceres
dR_EIS_hat_hist_ceres = dEIS_hat_hist * eis_obs_kernels.ceres
dR_Tadv_hat_hist_ceres = dTadv_hat_hist * tadv_obs_kernels.ceres
dR_RH_hat_hist_ceres = dRH_hat_hist * rh_obs_kernels.ceres
dR_Omega_hat_hist_ceres = dOmega_hat_hist * omega_obs_kernels.ceres
dR_WS_hat_hist_ceres = dWS_hat_hist * ws_obs_kernels.ceres

# Abrupt fast response
dR_SST_hat_fast_ceres = dSST_hat_fast * sst_obs_kernels.ceres
dR_EIS_hat_fast_ceres = dEIS_hat_fast * eis_obs_kernels.ceres
dR_Tadv_hat_fast_ceres = dTadv_hat_fast * tadv_obs_kernels.ceres
dR_RH_hat_fast_ceres = dRH_hat_fast * rh_obs_kernels.ceres
dR_Omega_hat_fast_ceres = dOmega_hat_fast * omega_obs_kernels.ceres
dR_WS_hat_fast_ceres = dWS_hat_fast * ws_obs_kernels.ceres

# Abrupt slow response
dR_SST_hat_slow_ceres = dSST_hat_slow * sst_obs_kernels.ceres
dR_EIS_hat_slow_ceres = dEIS_hat_slow * eis_obs_kernels.ceres
dR_Tadv_hat_slow_ceres = dTadv_hat_slow * tadv_obs_kernels.ceres
dR_RH_hat_slow_ceres = dRH_hat_slow * rh_obs_kernels.ceres
dR_Omega_hat_slow_ceres = dOmega_hat_slow * omega_obs_kernels.ceres
dR_WS_hat_slow_ceres = dWS_hat_slow * ws_obs_kernels.ceres


#Total low-cloud feedback
dR_hat_amip_ceres = dR_SST_hat_amip_ceres + dR_EIS_hat_amip_ceres + dR_Tadv_hat_amip_ceres + dR_RH_hat_amip_ceres + dR_Omega_hat_amip_ceres + dR_WS_hat_amip_ceres
dR_hat_fast_ceres = dR_SST_hat_fast_ceres + dR_EIS_hat_fast_ceres + dR_Tadv_hat_fast_ceres + dR_RH_hat_fast_ceres + dR_Omega_hat_fast_ceres + dR_WS_hat_fast_ceres
dR_hat_slow_ceres = dR_SST_hat_slow_ceres + dR_EIS_hat_slow_ceres + dR_Tadv_hat_slow_ceres + dR_RH_hat_slow_ceres + dR_Omega_hat_slow_ceres + dR_WS_hat_slow_ceres
dR_hat_hist_ceres = dR_SST_hat_hist_ceres + dR_EIS_hat_hist_ceres + dR_Tadv_hat_hist_ceres + dR_RH_hat_hist_ceres + dR_Omega_hat_hist_ceres + dR_WS_hat_hist_ceres

#### Patmosx

In [12]:
# Obs AMIP
dR_SST_hat_amip_patmosx= dSST_hat_amip * sst_obs_kernels.patmosx
dR_EIS_hat_amip_patmosx= dEIS_hat_amip * eis_obs_kernels.patmosx
dR_Tadv_hat_amip_patmosx= dTadv_hat_amip * tadv_obs_kernels.patmosx
dR_RH_hat_amip_patmosx= dRH_hat_amip * rh_obs_kernels.patmosx
dR_Omega_hat_amip_patmosx= dOmega_hat_amip * omega_obs_kernels.patmosx
dR_WS_hat_amip_patmosx= dWS_hat_amip * ws_obs_kernels.patmosx

# Historical
dR_SST_hat_hist_patmosx= dSST_hat_hist * sst_obs_kernels.patmosx
dR_EIS_hat_hist_patmosx= dEIS_hat_hist * eis_obs_kernels.patmosx
dR_Tadv_hat_hist_patmosx= dTadv_hat_hist * tadv_obs_kernels.patmosx
dR_RH_hat_hist_patmosx= dRH_hat_hist * rh_obs_kernels.patmosx
dR_Omega_hat_hist_patmosx= dOmega_hat_hist * omega_obs_kernels.patmosx
dR_WS_hat_hist_patmosx= dWS_hat_hist * ws_obs_kernels.patmosx


# Abrupt fast response
dR_SST_hat_fast_patmosx= dSST_hat_fast * sst_obs_kernels.patmosx
dR_EIS_hat_fast_patmosx= dEIS_hat_fast * eis_obs_kernels.patmosx
dR_Tadv_hat_fast_patmosx= dTadv_hat_fast * tadv_obs_kernels.patmosx
dR_RH_hat_fast_patmosx= dRH_hat_fast * rh_obs_kernels.patmosx
dR_Omega_hat_fast_patmosx= dOmega_hat_fast * omega_obs_kernels.patmosx
dR_WS_hat_fast_patmosx= dWS_hat_fast * ws_obs_kernels.patmosx

# Abrupt slow response
dR_SST_hat_slow_patmosx= dSST_hat_slow * sst_obs_kernels.patmosx
dR_EIS_hat_slow_patmosx= dEIS_hat_slow * eis_obs_kernels.patmosx
dR_Tadv_hat_slow_patmosx= dTadv_hat_slow * tadv_obs_kernels.patmosx
dR_RH_hat_slow_patmosx= dRH_hat_slow * rh_obs_kernels.patmosx
dR_Omega_hat_slow_patmosx= dOmega_hat_slow * omega_obs_kernels.patmosx
dR_WS_hat_slow_patmosx= dWS_hat_slow * ws_obs_kernels.patmosx

#Total low-cloud feedback
dR_hat_amip_patmosx= dR_SST_hat_amip_patmosx+ dR_EIS_hat_amip_patmosx+ dR_Tadv_hat_amip_patmosx+ dR_RH_hat_amip_patmosx+ dR_Omega_hat_amip_patmosx+ dR_WS_hat_amip_patmosx
dR_hat_fast_patmosx= dR_SST_hat_fast_patmosx+ dR_EIS_hat_fast_patmosx+ dR_Tadv_hat_fast_patmosx+ dR_RH_hat_fast_patmosx+ dR_Omega_hat_fast_patmosx+ dR_WS_hat_fast_patmosx
dR_hat_slow_patmosx= dR_SST_hat_slow_patmosx+ dR_EIS_hat_slow_patmosx+ dR_Tadv_hat_slow_patmosx+ dR_RH_hat_slow_patmosx+ dR_Omega_hat_slow_patmosx+ dR_WS_hat_slow_patmosx
dR_hat_hist_patmosx= dR_SST_hat_hist_patmosx+ dR_EIS_hat_hist_patmosx+ dR_Tadv_hat_hist_patmosx+ dR_RH_hat_hist_patmosx+ dR_Omega_hat_hist_patmosx+ dR_WS_hat_hist_patmosx

#### ISCCP

In [13]:
# Obs AMIP
dR_SST_hat_amip_isccp= dSST_hat_amip * sst_obs_kernels.isccp
dR_EIS_hat_amip_isccp= dEIS_hat_amip * eis_obs_kernels.isccp
dR_Tadv_hat_amip_isccp= dTadv_hat_amip * tadv_obs_kernels.isccp
dR_RH_hat_amip_isccp= dRH_hat_amip * rh_obs_kernels.isccp
dR_Omega_hat_amip_isccp= dOmega_hat_amip * omega_obs_kernels.isccp
dR_WS_hat_amip_isccp= dWS_hat_amip * ws_obs_kernels.isccp

# Historical
dR_SST_hat_hist_isccp= dSST_hat_hist * sst_obs_kernels.isccp
dR_EIS_hat_hist_isccp= dEIS_hat_hist * eis_obs_kernels.isccp
dR_Tadv_hat_hist_isccp= dTadv_hat_hist * tadv_obs_kernels.isccp
dR_RH_hat_hist_isccp= dRH_hat_hist * rh_obs_kernels.isccp
dR_Omega_hat_hist_isccp= dOmega_hat_hist * omega_obs_kernels.isccp
dR_WS_hat_hist_isccp= dWS_hat_hist * ws_obs_kernels.isccp


# Abrupt fast response
dR_SST_hat_fast_isccp= dSST_hat_fast * sst_obs_kernels.isccp
dR_EIS_hat_fast_isccp= dEIS_hat_fast * eis_obs_kernels.isccp
dR_Tadv_hat_fast_isccp= dTadv_hat_fast * tadv_obs_kernels.isccp
dR_RH_hat_fast_isccp= dRH_hat_fast * rh_obs_kernels.isccp
dR_Omega_hat_fast_isccp= dOmega_hat_fast * omega_obs_kernels.isccp
dR_WS_hat_fast_isccp= dWS_hat_fast * ws_obs_kernels.isccp

# Abrupt slow response
dR_SST_hat_slow_isccp= dSST_hat_slow * sst_obs_kernels.isccp
dR_EIS_hat_slow_isccp= dEIS_hat_slow * eis_obs_kernels.isccp
dR_Tadv_hat_slow_isccp= dTadv_hat_slow * tadv_obs_kernels.isccp
dR_RH_hat_slow_isccp= dRH_hat_slow * rh_obs_kernels.isccp
dR_Omega_hat_slow_isccp= dOmega_hat_slow * omega_obs_kernels.isccp
dR_WS_hat_slow_isccp= dWS_hat_slow * ws_obs_kernels.isccp

#Total low-cloud feedback
dR_hat_amip_isccp = dR_SST_hat_amip_isccp + dR_EIS_hat_amip_isccp + dR_Tadv_hat_amip_isccp + dR_RH_hat_amip_isccp + dR_Omega_hat_amip_isccp + dR_WS_hat_amip_isccp
dR_hat_fast_isccp = dR_SST_hat_fast_isccp + dR_EIS_hat_fast_isccp + dR_Tadv_hat_fast_isccp + dR_RH_hat_fast_isccp + dR_Omega_hat_fast_isccp + dR_WS_hat_fast_isccp
dR_hat_slow_isccp = dR_SST_hat_slow_isccp + dR_EIS_hat_slow_isccp + dR_Tadv_hat_slow_isccp + dR_RH_hat_slow_isccp + dR_Omega_hat_slow_isccp + dR_WS_hat_slow_isccp
dR_hat_hist_isccp = dR_SST_hat_hist_isccp + dR_EIS_hat_hist_isccp + dR_Tadv_hat_hist_isccp + dR_RH_hat_hist_isccp + dR_Omega_hat_hist_isccp + dR_WS_hat_hist_isccp


#### MODIS

In [14]:
# Obs AMIP
dR_SST_hat_amip_modis= dSST_hat_amip * sst_obs_kernels.modis
dR_EIS_hat_amip_modis= dEIS_hat_amip * eis_obs_kernels.modis
dR_Tadv_hat_amip_modis= dTadv_hat_amip * tadv_obs_kernels.modis
dR_RH_hat_amip_modis= dRH_hat_amip * rh_obs_kernels.modis
dR_Omega_hat_amip_modis= dOmega_hat_amip * omega_obs_kernels.modis
dR_WS_hat_amip_modis= dWS_hat_amip * ws_obs_kernels.modis

# Historical
dR_SST_hat_hist_modis= dSST_hat_hist * sst_obs_kernels.modis
dR_EIS_hat_hist_modis= dEIS_hat_hist * eis_obs_kernels.modis
dR_Tadv_hat_hist_modis= dTadv_hat_hist * tadv_obs_kernels.modis
dR_RH_hat_hist_modis= dRH_hat_hist * rh_obs_kernels.modis
dR_Omega_hat_hist_modis= dOmega_hat_hist * omega_obs_kernels.modis
dR_WS_hat_hist_modis= dWS_hat_hist * ws_obs_kernels.modis


# Abrupt fast response
dR_SST_hat_fast_modis= dSST_hat_fast * sst_obs_kernels.modis
dR_EIS_hat_fast_modis= dEIS_hat_fast * eis_obs_kernels.modis
dR_Tadv_hat_fast_modis= dTadv_hat_fast * tadv_obs_kernels.modis
dR_RH_hat_fast_modis= dRH_hat_fast * rh_obs_kernels.modis
dR_Omega_hat_fast_modis= dOmega_hat_fast * omega_obs_kernels.modis
dR_WS_hat_fast_modis= dWS_hat_fast * ws_obs_kernels.modis

# Abrupt slow response
dR_SST_hat_slow_modis= dSST_hat_slow * sst_obs_kernels.modis
dR_EIS_hat_slow_modis= dEIS_hat_slow * eis_obs_kernels.modis
dR_Tadv_hat_slow_modis= dTadv_hat_slow * tadv_obs_kernels.modis
dR_RH_hat_slow_modis= dRH_hat_slow * rh_obs_kernels.modis
dR_Omega_hat_slow_modis= dOmega_hat_slow * omega_obs_kernels.modis
dR_WS_hat_slow_modis= dWS_hat_slow * ws_obs_kernels.modis

#Total low-cloud feedback
dR_hat_amip_modis = dR_SST_hat_amip_modis + dR_EIS_hat_amip_modis + dR_Tadv_hat_amip_modis + dR_RH_hat_amip_modis + dR_Omega_hat_amip_modis + dR_WS_hat_amip_modis
dR_hat_fast_modis = dR_SST_hat_fast_modis + dR_EIS_hat_fast_modis + dR_Tadv_hat_fast_modis + dR_RH_hat_fast_modis + dR_Omega_hat_fast_modis + dR_WS_hat_fast_modis
dR_hat_slow_modis = dR_SST_hat_slow_modis + dR_EIS_hat_slow_modis + dR_Tadv_hat_slow_modis + dR_RH_hat_slow_modis + dR_Omega_hat_slow_modis + dR_WS_hat_slow_modis
dR_hat_hist_modis = dR_SST_hat_hist_modis + dR_EIS_hat_hist_modis + dR_Tadv_hat_hist_modis + dR_RH_hat_hist_modis + dR_Omega_hat_hist_modis + dR_WS_hat_hist_modis


####  kernels * dSST_bar

In [20]:
# AMIP
dR_SST_hat_bar_amip = dSST_hat_bar_amip * sst_kernels
dR_EIS_hat_bar_amip = dEIS_hat_bar_amip * eis_kernels
dR_Tadv_hat_bar_amip = dTadv_hat_bar_amip * tadv_kernels
dR_RH_hat_bar_amip = dRH_hat_bar_amip * rh_kernels
dR_Omega_hat_bar_amip = dOmega_hat_bar_amip * omega_kernels
dR_WS_hat_bar_amip = dWS_hat_bar_amip * ws_kernels

# Historical
dR_SST_hat_bar_hist = dSST_hat_bar_hist * sst_kernels
dR_EIS_hat_bar_hist = dEIS_hat_bar_hist * eis_kernels
dR_Tadv_hat_bar_hist = dTadv_hat_bar_hist * tadv_kernels
dR_RH_hat_bar_hist = dRH_hat_bar_hist * rh_kernels
dR_Omega_hat_bar_hist = dOmega_hat_bar_hist * omega_kernels
dR_WS_hat_bar_hist = dWS_hat_bar_hist * ws_kernels


# Abrupt fast response
dR_SST_hat_bar_fast = dSST_hat_bar_fast * sst_kernels
dR_EIS_hat_bar_fast = dEIS_hat_bar_fast * eis_kernels
dR_Tadv_hat_bar_fast = dTadv_hat_bar_fast * tadv_kernels
dR_RH_hat_bar_fast = dRH_hat_bar_fast * rh_kernels
dR_Omega_hat_bar_fast = dOmega_hat_bar_fast * omega_kernels
dR_WS_hat_bar_fast = dWS_hat_bar_fast * ws_kernels

# Abrupt slow response
dR_SST_hat_bar_slow = dSST_hat_bar_slow * sst_kernels
dR_EIS_hat_bar_slow = dEIS_hat_bar_slow * eis_kernels
dR_Tadv_hat_bar_slow = dTadv_hat_bar_slow * tadv_kernels
dR_RH_hat_bar_slow = dRH_hat_bar_slow * rh_kernels
dR_Omega_hat_bar_slow = dOmega_hat_bar_slow * omega_kernels
dR_WS_hat_bar_slow = dWS_hat_bar_slow * ws_kernels


#Total low-cloud feedback
dR_hat_bar_amip = dR_SST_hat_bar_amip + dR_EIS_hat_bar_amip + dR_Tadv_hat_bar_amip + dR_RH_hat_bar_amip + dR_Omega_hat_bar_amip + dR_WS_hat_bar_amip
dR_hat_bar_fast = dR_SST_hat_bar_fast + dR_EIS_hat_bar_fast + dR_Tadv_hat_bar_fast + dR_RH_hat_bar_fast + dR_Omega_hat_bar_fast + dR_WS_hat_bar_fast
dR_hat_bar_slow = dR_SST_hat_bar_slow + dR_EIS_hat_bar_slow + dR_Tadv_hat_bar_slow + dR_RH_hat_bar_slow + dR_Omega_hat_bar_slow + dR_WS_hat_bar_slow
dR_hat_bar_hist = dR_SST_hat_bar_hist + dR_EIS_hat_bar_hist + dR_Tadv_hat_bar_hist + dR_RH_hat_bar_hist + dR_Omega_hat_bar_hist + dR_WS_hat_bar_hist

#### kernel_bar * dCCF

In [19]:
# AMIP
dR_sst_kernel_bar_amip = dSST_hat_amip * sst_kernels_bar
dR_eis_kernel_bar_amip = dEIS_hat_amip * eis_kernels_bar
dR_tadv_kernel_bar_amip = dTadv_hat_amip * tadv_kernels_bar
dR_rh_kernel_bar_amip = dRH_hat_amip * rh_kernels_bar
dR_omega_kernel_bar_amip = dOmega_hat_amip * omega_kernels_bar
dR_ws_kernel_bar_amip = dWS_hat_amip * ws_kernels_bar

# Historical
dR_sst_kernel_bar_hist = dSST_hat_hist * sst_kernels_bar
dR_eis_kernel_bar_hist = dEIS_hat_hist * eis_kernels_bar
dR_tadv_kernel_bar_hist = dTadv_hat_hist * tadv_kernels_bar
dR_rh_kernel_bar_hist = dRH_hat_hist * rh_kernels_bar
dR_omega_kernel_bar_hist = dOmega_hat_hist * omega_kernels_bar
dR_ws_kernel_bar_hist = dWS_hat_hist * ws_kernels_bar


# Abrupt fast response
dR_sst_kernel_bar_fast = dSST_hat_fast * sst_kernels_bar
dR_eis_kernel_bar_fast = dEIS_hat_fast * eis_kernels_bar
dR_tadv_kernel_bar_fast = dTadv_hat_fast * tadv_kernels_bar
dR_rh_kernel_bar_fast = dRH_hat_fast * rh_kernels_bar
dR_omega_kernel_bar_fast = dOmega_hat_fast * omega_kernels_bar
dR_ws_kernel_bar_fast = dWS_hat_fast * ws_kernels_bar

# Abrupt slow response
dR_sst_kernel_bar_slow = dSST_hat_slow * sst_kernels_bar
dR_eis_kernel_bar_slow = dEIS_hat_slow * eis_kernels_bar
dR_tadv_kernel_bar_slow = dTadv_hat_slow * tadv_kernels_bar
dR_rh_kernel_bar_slow = dRH_hat_slow * rh_kernels_bar
dR_omega_kernel_bar_slow = dOmega_hat_slow * omega_kernels_bar
dR_ws_kernel_bar_slow = dWS_hat_slow * ws_kernels_bar

#Total low-cloud feedback
dR_hat_kernel_bar_amip = dR_sst_kernel_bar_amip + dR_eis_kernel_bar_amip + dR_tadv_kernel_bar_amip + dR_rh_kernel_bar_amip + dR_omega_kernel_bar_amip + dR_ws_kernel_bar_amip
dR_hat_kernel_bar_hist = dR_sst_kernel_bar_hist + dR_eis_kernel_bar_hist + dR_tadv_kernel_bar_hist + dR_rh_kernel_bar_hist + dR_omega_kernel_bar_hist + dR_ws_kernel_bar_hist
dR_hat_kernel_bar_fast = dR_sst_kernel_bar_fast + dR_eis_kernel_bar_fast + dR_tadv_kernel_bar_fast + dR_rh_kernel_bar_fast + dR_omega_kernel_bar_fast + dR_ws_kernel_bar_fast
dR_hat_kernel_bar_slow = dR_sst_kernel_bar_slow + dR_eis_kernel_bar_slow + dR_tadv_kernel_bar_slow + dR_rh_kernel_bar_slow + dR_omega_kernel_bar_slow + dR_ws_kernel_bar_slow

# Global Feedback and Feedback per CCF spread across Models 

### Models kernel * dCCF_hat

In [16]:
def to_dataframe(xrdataset,experiment,variable):
    globmean=xrdataset.mean('lat',skipna=True).mean('lon',skipna=True).to_array('models')
    globmodelmean=globmean.mean('models')
    df=pd.DataFrame(globmean.values, columns=['dR/dT'])
    df=df.append(pd.DataFrame([globmodelmean.values], columns=['dR/dT']))
    df.loc[:, 'models'] = modelname_list+['mean']
    df[['experiment','var']]=experiment,variable
    return df


df = to_dataframe(dR_hat_amip,'amip','Total')
df = df.append(to_dataframe(dR_hat_hist,'hist','Total'))
df = df.append(to_dataframe(dR_hat_fast,'fast','Total'))
df = df.append(to_dataframe(dR_hat_slow,'slow','Total'))
df = df.append(to_dataframe(dR_SST_hat_amip,'amip','SST'))
df = df.append(to_dataframe(dR_EIS_hat_amip,'amip','EIS'))
df = df.append(to_dataframe(dR_Tadv_hat_amip,'amip','Tadv'))
df = df.append(to_dataframe(dR_RH_hat_amip,'amip','RH'))
df = df.append(to_dataframe(dR_Omega_hat_amip,'amip','Omega'))
df = df.append(to_dataframe(dR_WS_hat_amip,'amip','WS'))
df = df.append(to_dataframe(dR_SST_hat_hist,'hist','SST'))
df = df.append(to_dataframe(dR_EIS_hat_hist,'hist','EIS'))
df = df.append(to_dataframe(dR_Tadv_hat_hist,'hist','Tadv'))
df = df.append(to_dataframe(dR_RH_hat_hist,'hist','RH'))
df = df.append(to_dataframe(dR_Omega_hat_hist,'hist','Omega'))
df = df.append(to_dataframe(dR_WS_hat_hist,'hist','WS'))
df = df.append(to_dataframe(dR_SST_hat_fast,'fast','SST'))
df = df.append(to_dataframe(dR_EIS_hat_fast,'fast','EIS'))
df = df.append(to_dataframe(dR_Tadv_hat_fast,'fast','Tadv'))
df = df.append(to_dataframe(dR_RH_hat_fast,'fast','RH'))
df = df.append(to_dataframe(dR_Omega_hat_fast,'fast','Omega'))
df = df.append(to_dataframe(dR_WS_hat_fast,'fast','WS'))
df = df.append(to_dataframe(dR_SST_hat_slow,'slow','SST'))
df = df.append(to_dataframe(dR_EIS_hat_slow,'slow','EIS'))
df = df.append(to_dataframe(dR_Tadv_hat_slow,'slow','Tadv'))
df = df.append(to_dataframe(dR_RH_hat_slow,'slow','RH'))
df = df.append(to_dataframe(dR_Omega_hat_slow,'slow','Omega'))
df = df.append(to_dataframe(dR_WS_hat_slow,'slow','WS'))


KeyboardInterrupt



## Obs kernels 

In [ ]:
df_ceres = to_dataframe(dR_hat_amip_ceres,'amip','Total')
df_ceres = df_ceres.append(to_dataframe(dR_hat_hist_ceres,'hist','Total'))
df_ceres = df_ceres.append(to_dataframe(dR_hat_fast_ceres,'fast','Total'))
df_ceres = df_ceres.append(to_dataframe(dR_hat_slow_ceres,'slow','Total'))
df_ceres = df_ceres.append(to_dataframe(dR_SST_hat_amip_ceres,'amip','SST'))
df_ceres = df_ceres.append(to_dataframe(dR_EIS_hat_amip_ceres,'amip','EIS'))
df_ceres = df_ceres.append(to_dataframe(dR_Tadv_hat_amip_ceres,'amip','Tadv'))
df_ceres = df_ceres.append(to_dataframe(dR_RH_hat_amip_ceres,'amip','RH'))
df_ceres = df_ceres.append(to_dataframe(dR_Omega_hat_amip_ceres,'amip','Omega'))
df_ceres = df_ceres.append(to_dataframe(dR_WS_hat_amip_ceres,'amip','WS'))
df_ceres = df_ceres.append(to_dataframe(dR_SST_hat_hist_ceres,'hist','SST'))
df_ceres = df_ceres.append(to_dataframe(dR_EIS_hat_hist_ceres,'hist','EIS'))
df_ceres = df_ceres.append(to_dataframe(dR_Tadv_hat_hist_ceres,'hist','Tadv'))
df_ceres = df_ceres.append(to_dataframe(dR_RH_hat_hist_ceres,'hist','RH'))
df_ceres = df_ceres.append(to_dataframe(dR_Omega_hat_hist_ceres,'hist','Omega'))
df_ceres = df_ceres.append(to_dataframe(dR_WS_hat_hist_ceres,'hist','WS'))
df_ceres = df_ceres.append(to_dataframe(dR_SST_hat_fast_ceres,'fast','SST'))
df_ceres = df_ceres.append(to_dataframe(dR_EIS_hat_fast_ceres,'fast','EIS'))
df_ceres = df_ceres.append(to_dataframe(dR_Tadv_hat_fast_ceres,'fast','Tadv'))
df_ceres = df_ceres.append(to_dataframe(dR_RH_hat_fast_ceres,'fast','RH'))
df_ceres = df_ceres.append(to_dataframe(dR_Omega_hat_fast_ceres,'fast','Omega'))
df_ceres = df_ceres.append(to_dataframe(dR_WS_hat_fast_ceres,'fast','WS'))
df_ceres = df_ceres.append(to_dataframe(dR_SST_hat_slow_ceres,'slow','SST'))
df_ceres = df_ceres.append(to_dataframe(dR_EIS_hat_slow_ceres,'slow','EIS'))
df_ceres = df_ceres.append(to_dataframe(dR_Tadv_hat_slow_ceres,'slow','Tadv'))
df_ceres = df_ceres.append(to_dataframe(dR_RH_hat_slow_ceres,'slow','RH'))
df_ceres = df_ceres.append(to_dataframe(dR_Omega_hat_slow_ceres,'slow','Omega'))
df_ceres = df_ceres.append(to_dataframe(dR_WS_hat_slow_ceres,'slow','WS'))

In [ ]:
df_patmosx = to_dataframe(dR_hat_amip_patmosx,'amip','Total')
df_patmosx = df_patmosx.append(to_dataframe(dR_hat_hist_patmosx,'hist','Total'))
df_patmosx = df_patmosx.append(to_dataframe(dR_hat_fast_patmosx,'fast','Total'))
df_patmosx = df_patmosx.append(to_dataframe(dR_hat_slow_patmosx,'slow','Total'))
df_patmosx = df_patmosx.append(to_dataframe(dR_SST_hat_amip_patmosx,'amip','SST'))
df_patmosx = df_patmosx.append(to_dataframe(dR_EIS_hat_amip_patmosx,'amip','EIS'))
df_patmosx = df_patmosx.append(to_dataframe(dR_Tadv_hat_amip_patmosx,'amip','Tadv'))
df_patmosx = df_patmosx.append(to_dataframe(dR_RH_hat_amip_patmosx,'amip','RH'))
df_patmosx = df_patmosx.append(to_dataframe(dR_Omega_hat_amip_patmosx,'amip','Omega'))
df_patmosx = df_patmosx.append(to_dataframe(dR_WS_hat_amip_patmosx,'amip','WS'))
df_patmosx = df_patmosx.append(to_dataframe(dR_SST_hat_hist_patmosx,'hist','SST'))
df_patmosx = df_patmosx.append(to_dataframe(dR_EIS_hat_hist_patmosx,'hist','EIS'))
df_patmosx = df_patmosx.append(to_dataframe(dR_Tadv_hat_hist_patmosx,'hist','Tadv'))
df_patmosx = df_patmosx.append(to_dataframe(dR_RH_hat_hist_patmosx,'hist','RH'))
df_patmosx = df_patmosx.append(to_dataframe(dR_Omega_hat_hist_patmosx,'hist','Omega'))
df_patmosx = df_patmosx.append(to_dataframe(dR_WS_hat_hist_patmosx,'hist','WS'))
df_patmosx = df_patmosx.append(to_dataframe(dR_SST_hat_fast_patmosx,'fast','SST'))
df_patmosx = df_patmosx.append(to_dataframe(dR_EIS_hat_fast_patmosx,'fast','EIS'))
df_patmosx = df_patmosx.append(to_dataframe(dR_Tadv_hat_fast_patmosx,'fast','Tadv'))
df_patmosx = df_patmosx.append(to_dataframe(dR_RH_hat_fast_patmosx,'fast','RH'))
df_patmosx = df_patmosx.append(to_dataframe(dR_Omega_hat_fast_patmosx,'fast','Omega'))
df_patmosx = df_patmosx.append(to_dataframe(dR_WS_hat_fast_patmosx,'fast','WS'))
df_patmosx = df_patmosx.append(to_dataframe(dR_SST_hat_slow_patmosx,'slow','SST'))
df_patmosx = df_patmosx.append(to_dataframe(dR_EIS_hat_slow_patmosx,'slow','EIS'))
df_patmosx = df_patmosx.append(to_dataframe(dR_Tadv_hat_slow_patmosx,'slow','Tadv'))
df_patmosx = df_patmosx.append(to_dataframe(dR_RH_hat_slow_patmosx,'slow','RH'))
df_patmosx = df_patmosx.append(to_dataframe(dR_Omega_hat_slow_patmosx,'slow','Omega'))
df_patmosx = df_patmosx.append(to_dataframe(dR_WS_hat_slow_patmosx,'slow','WS'))

In [ ]:
df_isccp = to_dataframe(dR_hat_amip_isccp,'amip','Total')
df_isccp = df_isccp.append(to_dataframe(dR_hat_hist_isccp,'hist','Total'))
df_isccp = df_isccp.append(to_dataframe(dR_hat_fast_isccp,'fast','Total'))
df_isccp = df_isccp.append(to_dataframe(dR_hat_slow_isccp,'slow','Total'))
df_isccp = df_isccp.append(to_dataframe(dR_SST_hat_amip_isccp,'amip','SST'))
df_isccp = df_isccp.append(to_dataframe(dR_EIS_hat_amip_isccp,'amip','EIS'))
df_isccp = df_isccp.append(to_dataframe(dR_Tadv_hat_amip_isccp,'amip','Tadv'))
df_isccp = df_isccp.append(to_dataframe(dR_RH_hat_amip_isccp,'amip','RH'))
df_isccp = df_isccp.append(to_dataframe(dR_Omega_hat_amip_isccp,'amip','Omega'))
df_isccp = df_isccp.append(to_dataframe(dR_WS_hat_amip_isccp,'amip','WS'))
df_isccp = df_isccp.append(to_dataframe(dR_SST_hat_hist_isccp,'hist','SST'))
df_isccp = df_isccp.append(to_dataframe(dR_EIS_hat_hist_isccp,'hist','EIS'))
df_isccp = df_isccp.append(to_dataframe(dR_Tadv_hat_hist_isccp,'hist','Tadv'))
df_isccp = df_isccp.append(to_dataframe(dR_RH_hat_hist_isccp,'hist','RH'))
df_isccp = df_isccp.append(to_dataframe(dR_Omega_hat_hist_isccp,'hist','Omega'))
df_isccp = df_isccp.append(to_dataframe(dR_WS_hat_hist_isccp,'hist','WS'))
df_isccp = df_isccp.append(to_dataframe(dR_SST_hat_fast_isccp,'fast','SST'))
df_isccp = df_isccp.append(to_dataframe(dR_EIS_hat_fast_isccp,'fast','EIS'))
df_isccp = df_isccp.append(to_dataframe(dR_Tadv_hat_fast_isccp,'fast','Tadv'))
df_isccp = df_isccp.append(to_dataframe(dR_RH_hat_fast_isccp,'fast','RH'))
df_isccp = df_isccp.append(to_dataframe(dR_Omega_hat_fast_isccp,'fast','Omega'))
df_isccp = df_isccp.append(to_dataframe(dR_WS_hat_fast_isccp,'fast','WS'))
df_isccp = df_isccp.append(to_dataframe(dR_SST_hat_slow_isccp,'slow','SST'))
df_isccp = df_isccp.append(to_dataframe(dR_EIS_hat_slow_isccp,'slow','EIS'))
df_isccp = df_isccp.append(to_dataframe(dR_Tadv_hat_slow_isccp,'slow','Tadv'))
df_isccp = df_isccp.append(to_dataframe(dR_RH_hat_slow_isccp,'slow','RH'))
df_isccp = df_isccp.append(to_dataframe(dR_Omega_hat_slow_isccp,'slow','Omega'))
df_isccp = df_isccp.append(to_dataframe(dR_WS_hat_slow_isccp,'slow','WS'))

In [ ]:
df_modis = to_dataframe(dR_hat_amip_modis,'amip','Total')
df_modis = df_modis.append(to_dataframe(dR_hat_hist_modis,'hist','Total'))
df_modis = df_modis.append(to_dataframe(dR_hat_fast_modis,'fast','Total'))
df_modis = df_modis.append(to_dataframe(dR_hat_slow_modis,'slow','Total'))
df_modis = df_modis.append(to_dataframe(dR_SST_hat_amip_modis,'amip','SST'))
df_modis = df_modis.append(to_dataframe(dR_EIS_hat_amip_modis,'amip','EIS'))
df_modis = df_modis.append(to_dataframe(dR_Tadv_hat_amip_modis,'amip','Tadv'))
df_modis = df_modis.append(to_dataframe(dR_RH_hat_amip_modis,'amip','RH'))
df_modis = df_modis.append(to_dataframe(dR_Omega_hat_amip_modis,'amip','Omega'))
df_modis = df_modis.append(to_dataframe(dR_WS_hat_amip_modis,'amip','WS'))
df_modis = df_modis.append(to_dataframe(dR_SST_hat_hist_modis,'hist','SST'))
df_modis = df_modis.append(to_dataframe(dR_EIS_hat_hist_modis,'hist','EIS'))
df_modis = df_modis.append(to_dataframe(dR_Tadv_hat_hist_modis,'hist','Tadv'))
df_modis = df_modis.append(to_dataframe(dR_RH_hat_hist_modis,'hist','RH'))
df_modis = df_modis.append(to_dataframe(dR_Omega_hat_hist_modis,'hist','Omega'))
df_modis = df_modis.append(to_dataframe(dR_WS_hat_hist_modis,'hist','WS'))
df_modis = df_modis.append(to_dataframe(dR_SST_hat_fast_modis,'fast','SST'))
df_modis = df_modis.append(to_dataframe(dR_EIS_hat_fast_modis,'fast','EIS'))
df_modis = df_modis.append(to_dataframe(dR_Tadv_hat_fast_modis,'fast','Tadv'))
df_modis = df_modis.append(to_dataframe(dR_RH_hat_fast_modis,'fast','RH'))
df_modis = df_modis.append(to_dataframe(dR_Omega_hat_fast_modis,'fast','Omega'))
df_modis = df_modis.append(to_dataframe(dR_WS_hat_fast_modis,'fast','WS'))
df_modis = df_modis.append(to_dataframe(dR_SST_hat_slow_modis,'slow','SST'))
df_modis = df_modis.append(to_dataframe(dR_EIS_hat_slow_modis,'slow','EIS'))
df_modis = df_modis.append(to_dataframe(dR_Tadv_hat_slow_modis,'slow','Tadv'))
df_modis = df_modis.append(to_dataframe(dR_RH_hat_slow_modis,'slow','RH'))
df_modis = df_modis.append(to_dataframe(dR_Omega_hat_slow_modis,'slow','Omega'))
df_modis = df_modis.append(to_dataframe(dR_WS_hat_slow_modis,'slow','WS'))

## Model kernels * dCCF_modelavg

In [ ]:
df_ccfavg = to_dataframe(dR_hat_bar_amip,'amip','Total')
df_ccfavg = df_ccfavg.append(to_dataframe(dR_hat_bar_hist,'hist','Total'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_hat_bar_fast,'fast','Total'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_hat_bar_slow,'slow','Total'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_SST_hat_bar_amip,'amip','SST'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_EIS_hat_bar_amip,'amip','EIS'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_Tadv_hat_bar_amip,'amip','Tadv'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_RH_hat_bar_amip,'amip','RH'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_Omega_hat_bar_amip,'amip','Omega'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_WS_hat_bar_amip,'amip','WS'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_SST_hat_bar_hist,'hist','SST'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_EIS_hat_bar_hist,'hist','EIS'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_Tadv_hat_bar_hist,'hist','Tadv'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_RH_hat_bar_hist,'hist','RH'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_Omega_hat_bar_hist,'hist','Omega'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_WS_hat_bar_hist,'hist','WS'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_SST_hat_bar_fast,'fast','SST'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_EIS_hat_bar_fast,'fast','EIS'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_Tadv_hat_bar_fast,'fast','Tadv'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_RH_hat_bar_fast,'fast','RH'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_Omega_hat_bar_fast,'fast','Omega'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_WS_hat_bar_fast,'fast','WS'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_SST_hat_bar_slow,'slow','SST'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_EIS_hat_bar_slow,'slow','EIS'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_Tadv_hat_bar_slow,'slow','Tadv'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_RH_hat_bar_slow,'slow','RH'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_Omega_hat_bar_slow,'slow','Omega'))
df_ccfavg = df_ccfavg.append(to_dataframe(dR_WS_hat_bar_slow,'slow','WS'))

# Kernel_modelavg * dCCF

In [21]:
df_kernelavg = to_dataframe(dR_hat_kernel_bar_amip,'amip','Total')
df_kernelavg = df_kernelavg.append(to_dataframe(dR_hat_kernel_bar_hist,'hist','Total'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_hat_kernel_bar_fast,'fast','Total'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_hat_kernel_bar_slow,'slow','Total'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_sst_kernel_bar_amip,'amip','SST'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_eis_kernel_bar_amip,'amip','EIS'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_tadv_kernel_bar_amip,'amip','Tadv'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_rh_kernel_bar_amip,'amip','RH'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_omega_kernel_bar_amip,'amip','Omega'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_ws_kernel_bar_amip,'amip','WS'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_sst_kernel_bar_hist,'hist','SST'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_eis_kernel_bar_hist,'hist','EIS'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_tadv_kernel_bar_hist,'hist','Tadv'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_rh_kernel_bar_hist,'hist','RH'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_omega_kernel_bar_hist,'hist','Omega'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_ws_kernel_bar_hist,'hist','WS'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_sst_kernel_bar_fast,'fast','SST'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_eis_kernel_bar_fast,'fast','EIS'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_tadv_kernel_bar_fast,'fast','Tadv'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_rh_kernel_bar_fast,'fast','RH'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_omega_kernel_bar_fast,'fast','Omega'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_ws_kernel_bar_fast,'fast','WS'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_sst_kernel_bar_slow,'slow','SST'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_eis_kernel_bar_slow,'slow','EIS'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_tadv_kernel_bar_slow,'slow','Tadv'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_rh_kernel_bar_slow,'slow','RH'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_omega_kernel_bar_slow,'slow','Omega'))
df_kernelavg = df_kernelavg.append(to_dataframe(dR_ws_kernel_bar_slow,'slow','WS'))

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarnin

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarnin

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarnin

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarnin

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarnin

...

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarnin

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/tmp/ipykernel_18159/3790598812.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(pd.DataFrame([globmodelmean.values], columns=['dR/dT']))
/tmp/ipykernel_18159/2257633932.py:25

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarnin

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarnin

/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarning: Polyfit may be poorly conditioned
  warn_on_deficient_rank(rank, x.shape[1])
/data/keeling/a/rytam2/miniconda3/lib/python3.9/site-packages/xarray/core/nputils.py:171: RankWarnin

In [23]:
df.to_csv('df.csv')
df_ccfavg.to_csv('df_ccfavg.csv')
df_ceres.to_csv('df_ceres.csv')
df_isccp.to_csv('df_isccp.csv')
df_modis.to_csv('df_modis.csv')
df_patmosx.to_csv('df_patmosx.csv')
df_kernelavg.to_csv('df_kernelavg.csv')

### Compiled

In [ ]:
import matplotlib as mpl

f = mpl.figure.Figure(figsize=(11, 5), )
sf1, sf2, sf3 = f.subfigures(3,1)
 
p1.on(sf1).save('/data/keeling/a/rytam2/ccf_model_spread/notebooks/figures/comp/dotplot_dRg_comp.png',  dpi=400, bbox_inches='tight')
p2.on(sf2).save('/data/keeling/a/rytam2/ccf_model_spread/notebooks/figures/comp/dotplot_dRg_comp.png',  dpi=400, bbox_inches='tight')
p6.on(sf3).save('/data/keeling/a/rytam2/ccf_model_spread/notebooks/figures/comp/dotplot_dRg_comp.png',  dpi=400, bbox_inches='tight')

# Uncertainty Analysis per CCF using Taylor Approximation

In [ ]:
'''# Kernel Ensemble Average
sst_kernels_bar = ensemble_mean(sst_kernels)
eis_kernels_bar = ensemble_mean(eis_kernels)
tadv_kernels_bar = ensemble_mean(tadv_kernels)
rh_kernels_bar = ensemble_mean(rh_kernels)
omega_kernels_bar = ensemble_mean(omega_kernels)
ws_kernels_bar = ensemble_mean(ws_kernels)'''

#Model deviations for AMIP 
dEIS_prime_amip = dEIS_hat_amip - dEIS_hat_bar_amip
dTadv_prime_amip = dTadv_hat_amip - dTadv_hat_bar_amip
dRH_prime_amip = dRH_hat_amip - dRH_hat_bar_amip
dOmega_prime_amip = dOmega_hat_amip - dOmega_hat_bar_amip
dWS_prime_amip = dWS_hat_amip - dWS_hat_bar_amip

#Model deviations for Historical
dEIS_prime_hist = dEIS_hat_hist - dEIS_hat_bar_hist
dTadv_prime_hist = dTadv_hat_hist - dTadv_hat_bar_hist
dRH_prime_hist = dRH_hat_hist - dRH_hat_bar_hist
dOmega_prime_hist = dOmega_hat_hist - dOmega_hat_bar_hist
dWS_prime_hist = dWS_hat_hist - dWS_hat_bar_hist

#Model deviations for Abrupt-4xCO2 fast response 
dEIS_prime_fast = dEIS_hat_fast - dEIS_hat_bar_fast
dTadv_prime_fast = dTadv_hat_fast - dTadv_hat_bar_fast
dRH_prime_fast = dRH_hat_fast - dRH_hat_bar_fast
dOmega_prime_fast = dOmega_hat_fast - dOmega_hat_bar_fast
dWS_prime_fast = dWS_hat_fast - dWS_hat_bar_fast

#Model deviations for Abrupt-4xCO2 slow response 
dEIS_prime_slow = dEIS_hat_slow - dEIS_hat_bar_slow
dTadv_prime_slow = dTadv_hat_slow - dTadv_hat_bar_slow
dRH_prime_slow = dRH_hat_slow - dRH_hat_bar_slow
dOmega_prime_slow = dOmega_hat_slow - dOmega_hat_bar_slow
dWS_prime_slow = dWS_hat_slow - dWS_hat_bar_slow



# kernel deviation from mean 
sst_kernels_prime = sst_kernels-sst_kernels_bar
eis_kernels_prime = eis_kernels-eis_kernels_bar
tadv_kernels_prime = tadv_kernels-tadv_kernels_bar
rh_kernels_prime = rh_kernels-rh_kernels_bar
omega_kernels_prime = omega_kernels-omega_kernels_bar
ws_kernels_prime = ws_kernels-ws_kernels_bar

In [ ]:
# Model SST Sensitivity
dR_model_to_sst_prime_amip = sst_kernels_prime*dSST_hat_bar_amip
dR_model_to_sst_prime_var_amip = dR_model_to_sst_prime_amip.to_array(dim='models').var('models')
dR_model_to_sst_prime_fast = sst_kernels_prime*dSST_hat_bar_fast
dR_model_to_sst_prime_var_fast = dR_model_to_sst_prime_fast.to_array(dim='models').var('models')
dR_model_to_sst_prime_slow = sst_kernels_prime*dSST_hat_bar_slow
dR_model_to_sst_prime_var_slow = dR_model_to_sst_prime_slow.to_array(dim='models').var('models')
dR_model_to_sst_prime_hist = sst_kernels_prime*dSST_hat_bar_hist
dR_model_to_sst_prime_var_hist = dR_model_to_sst_prime_hist.to_array(dim='models').var('models')

# Intrinsic EIS changes 
dR_intrin_eis_prime_amip = (eis_kernels_bar*dEIS_prime_amip)
dR_intrin_eis_prime_var_amip = dR_intrin_eis_prime_amip.to_array(dim='models').var('models')
dR_intrin_eis_prime_fast = (eis_kernels_bar*dEIS_prime_fast)
dR_intrin_eis_prime_var_fast = dR_intrin_eis_prime_fast.to_array(dim='models').var('models')
dR_intrin_eis_prime_slow = (eis_kernels_bar*dEIS_prime_slow)
dR_intrin_eis_prime_var_slow = dR_intrin_eis_prime_slow.to_array(dim='models').var('models')
dR_intrin_eis_prime_hist = (eis_kernels_bar*dEIS_prime_hist)
dR_intrin_eis_prime_var_hist = dR_intrin_eis_prime_hist.to_array(dim='models').var('models')

# Model EIS Sensitivity 
dR_model_to_eis_prime_amip = eis_kernels_prime*dEIS_hat_bar_amip
dR_model_to_eis_prime_var_amip = dR_model_to_eis_prime_amip.to_array(dim='models').var('models')
dR_model_to_eis_prime_fast = eis_kernels_prime*dEIS_hat_bar_fast
dR_model_to_eis_prime_var_fast = dR_model_to_eis_prime_fast.to_array(dim='models').var('models')
dR_model_to_eis_prime_slow = eis_kernels_prime*dEIS_hat_bar_slow
dR_model_to_eis_prime_var_slow = dR_model_to_eis_prime_slow.to_array(dim='models').var('models')
dR_model_to_eis_prime_hist = eis_kernels_prime*dEIS_hat_bar_hist
dR_model_to_eis_prime_var_hist = dR_model_to_eis_prime_hist.to_array(dim='models').var('models')

# Intrinsic Tadv changes 
dR_intrin_tadv_prime_amip = (tadv_kernels_bar*dTadv_prime_amip)
dR_intrin_tadv_prime_var_amip = dR_intrin_tadv_prime_amip.to_array(dim='models').var('models')
dR_intrin_tadv_prime_fast = (tadv_kernels_bar*dTadv_prime_fast)
dR_intrin_tadv_prime_var_fast = dR_intrin_tadv_prime_fast.to_array(dim='models').var('models')
dR_intrin_tadv_prime_slow = (tadv_kernels_bar*dTadv_prime_slow)
dR_intrin_tadv_prime_var_slow = dR_intrin_tadv_prime_slow.to_array(dim='models').var('models')
dR_intrin_tadv_prime_hist = (tadv_kernels_bar*dTadv_prime_hist)
dR_intrin_tadv_prime_var_hist = dR_intrin_tadv_prime_hist.to_array(dim='models').var('models')

# Model Tadv Sensitivity 
dR_model_to_tadv_prime_amip = tadv_kernels_prime*dTadv_hat_bar_amip
dR_model_to_tadv_prime_var_amip = dR_model_to_tadv_prime_amip.to_array(dim='models').var('models')
dR_model_to_tadv_prime_fast = tadv_kernels_prime*dTadv_hat_bar_fast
dR_model_to_tadv_prime_var_fast = dR_model_to_tadv_prime_fast.to_array(dim='models').var('models')
dR_model_to_tadv_prime_slow = tadv_kernels_prime*dTadv_hat_bar_slow
dR_model_to_tadv_prime_var_slow = dR_model_to_tadv_prime_slow.to_array(dim='models').var('models')
dR_model_to_tadv_prime_hist = tadv_kernels_prime*dTadv_hat_bar_hist
dR_model_to_tadv_prime_var_hist = dR_model_to_tadv_prime_hist.to_array(dim='models').var('models')

# Intrinsic RH changes 
dR_intrin_rh_prime_amip = (rh_kernels_bar*dRH_prime_amip)
dR_intrin_rh_prime_var_amip = dR_intrin_rh_prime_amip.to_array(dim='models').var('models')
dR_intrin_rh_prime_fast = (rh_kernels_bar*dRH_prime_fast)
dR_intrin_rh_prime_var_fast = dR_intrin_rh_prime_fast.to_array(dim='models').var('models')
dR_intrin_rh_prime_slow = (rh_kernels_bar*dRH_prime_slow)
dR_intrin_rh_prime_var_slow = dR_intrin_rh_prime_slow.to_array(dim='models').var('models')
dR_intrin_rh_prime_hist = (rh_kernels_bar*dRH_prime_hist)
dR_intrin_rh_prime_var_hist = dR_intrin_rh_prime_hist.to_array(dim='models').var('models')

# Model RH Sensitivity 
dR_model_to_rh_prime_amip = rh_kernels_prime*dRH_hat_bar_amip
dR_model_to_rh_prime_var_amip = dR_model_to_rh_prime_amip.to_array(dim='models').var('models')
dR_model_to_rh_prime_fast = rh_kernels_prime*dRH_hat_bar_fast
dR_model_to_rh_prime_var_fast = dR_model_to_rh_prime_fast.to_array(dim='models').var('models')
dR_model_to_rh_prime_slow = rh_kernels_prime*dRH_hat_bar_slow
dR_model_to_rh_prime_var_slow = dR_model_to_rh_prime_slow.to_array(dim='models').var('models')
dR_model_to_rh_prime_hist = rh_kernels_prime*dRH_hat_bar_hist
dR_model_to_rh_prime_var_hist = dR_model_to_rh_prime_hist.to_array(dim='models').var('models')

# Intrinsic Omega changes 
dR_intrin_omega_prime_amip = (omega_kernels_bar*dOmega_prime_amip)
dR_intrin_omega_prime_var_amip = dR_intrin_omega_prime_amip.to_array(dim='models').var('models')
dR_intrin_omega_prime_fast = (omega_kernels_bar*dOmega_prime_fast)
dR_intrin_omega_prime_var_fast = dR_intrin_omega_prime_fast.to_array(dim='models').var('models')
dR_intrin_omega_prime_slow = (omega_kernels_bar*dOmega_prime_slow)
dR_intrin_omega_prime_var_slow = dR_intrin_omega_prime_slow.to_array(dim='models').var('models')
dR_intrin_omega_prime_hist = (omega_kernels_bar*dOmega_prime_hist)
dR_intrin_omega_prime_var_hist = dR_intrin_omega_prime_hist.to_array(dim='models').var('models')

# Model Omega Sensitivity 
dR_model_to_omega_prime_amip = omega_kernels_prime*dOmega_hat_bar_amip
dR_model_to_omega_prime_var_amip = dR_model_to_omega_prime_amip.to_array(dim='models').var('models')
dR_model_to_omega_prime_fast = omega_kernels_prime*dOmega_hat_bar_fast
dR_model_to_omega_prime_var_fast = dR_model_to_omega_prime_fast.to_array(dim='models').var('models')
dR_model_to_omega_prime_slow = omega_kernels_prime*dOmega_hat_bar_slow
dR_model_to_omega_prime_var_slow = dR_model_to_omega_prime_slow.to_array(dim='models').var('models')
dR_model_to_omega_prime_hist = omega_kernels_prime*dOmega_hat_bar_hist
dR_model_to_omega_prime_var_hist = dR_model_to_omega_prime_hist.to_array(dim='models').var('models')

# Intrinsic WS changes 
dR_intrin_ws_prime_amip = (ws_kernels_bar*dWS_prime_amip)
dR_intrin_ws_prime_var_amip = dR_intrin_ws_prime_amip.to_array(dim='models').var('models')
dR_intrin_ws_prime_fast = (ws_kernels_bar*dWS_prime_fast)
dR_intrin_ws_prime_var_fast = dR_intrin_ws_prime_fast.to_array(dim='models').var('models')
dR_intrin_ws_prime_slow = (ws_kernels_bar*dWS_prime_slow)
dR_intrin_ws_prime_var_slow = dR_intrin_ws_prime_slow.to_array(dim='models').var('models')
dR_intrin_ws_prime_hist = (ws_kernels_bar*dWS_prime_hist)
dR_intrin_ws_prime_var_hist = dR_intrin_ws_prime_hist.to_array(dim='models').var('models')

# Model WS Sensitivity 
dR_model_to_ws_prime_amip = ws_kernels_prime*dWS_hat_bar_amip
dR_model_to_ws_prime_var_amip = dR_model_to_ws_prime_amip.to_array(dim='models').var('models')
dR_model_to_ws_prime_fast = ws_kernels_prime*dWS_hat_bar_fast
dR_model_to_ws_prime_var_fast = dR_model_to_ws_prime_fast.to_array(dim='models').var('models')
dR_model_to_ws_prime_slow = ws_kernels_prime*dWS_hat_bar_slow
dR_model_to_ws_prime_var_slow = dR_model_to_ws_prime_slow.to_array(dim='models').var('models')
dR_model_to_ws_prime_hist = ws_kernels_prime*dWS_hat_bar_hist
dR_model_to_ws_prime_var_hist = dR_model_to_ws_prime_hist.to_array(dim='models').var('models')



# Total Variance 
dR_intrin_var_amip = dR_intrin_eis_prime_var_amip+dR_intrin_tadv_prime_var_amip+dR_intrin_rh_prime_var_amip+dR_intrin_omega_prime_var_amip+dR_intrin_ws_prime_var_amip
dR_sensitivity_var_amip = dR_model_to_sst_prime_var_amip+dR_model_to_eis_prime_var_amip+dR_model_to_tadv_prime_var_amip+dR_model_to_rh_prime_var_amip+dR_model_to_omega_prime_var_amip+dR_model_to_ws_prime_var_amip
dR_hat_var_amip = dR_hat_amip.to_array('models').var('models')
var_ratio_amip = (dR_intrin_var_amip+dR_sensitivity_var_amip)/dR_hat_var_amip
var_diff_amip = dR_hat_var_amip-(dR_intrin_var_amip+dR_sensitivity_var_amip)

dR_intrin_var_fast = dR_intrin_eis_prime_var_fast+dR_intrin_tadv_prime_var_fast+dR_intrin_rh_prime_var_fast+dR_intrin_omega_prime_var_fast+dR_intrin_ws_prime_var_fast
dR_sensitivity_var_fast = dR_model_to_sst_prime_var_fast+dR_model_to_eis_prime_var_fast+dR_model_to_tadv_prime_var_fast+dR_model_to_rh_prime_var_fast+dR_model_to_omega_prime_var_fast+dR_model_to_ws_prime_var_fast
dR_hat_var_fast = dR_hat_fast.to_array('models').var('models')
var_ratio_fast = (dR_intrin_var_fast+dR_sensitivity_var_fast)/dR_hat_var_fast
var_diff_fast = dR_hat_var_fast-(dR_intrin_var_fast+dR_sensitivity_var_fast)

dR_intrin_var_slow = dR_intrin_eis_prime_var_slow+dR_intrin_tadv_prime_var_slow+dR_intrin_rh_prime_var_slow+dR_intrin_omega_prime_var_slow+dR_intrin_ws_prime_var_slow
dR_sensitivity_var_slow = dR_model_to_sst_prime_var_slow+dR_model_to_eis_prime_var_slow+dR_model_to_tadv_prime_var_slow+dR_model_to_rh_prime_var_slow+dR_model_to_omega_prime_var_slow+dR_model_to_ws_prime_var_slow
dR_hat_var_slow = dR_hat_slow.to_array('models').var('models')
var_ratio_slow = (dR_intrin_var_slow+dR_sensitivity_var_slow)/dR_hat_var_slow
var_diff_slow = dR_hat_var_slow-(dR_intrin_var_slow+dR_sensitivity_var_slow)

dR_intrin_var_hist = dR_intrin_eis_prime_var_hist+dR_intrin_tadv_prime_var_hist+dR_intrin_rh_prime_var_hist+dR_intrin_omega_prime_var_hist+dR_intrin_ws_prime_var_hist
dR_sensitivity_var_hist = dR_model_to_sst_prime_var_hist+dR_model_to_eis_prime_var_hist+dR_model_to_tadv_prime_var_hist+dR_model_to_rh_prime_var_hist+dR_model_to_omega_prime_var_hist+dR_model_to_ws_prime_var_hist
dR_hat_var_hist = dR_hat_hist.to_array('models').var('models')
var_ratio_hist = (dR_intrin_var_hist+dR_sensitivity_var_hist)/dR_hat_var_hist
var_diff_hist = dR_hat_var_hist-(dR_intrin_var_hist+dR_sensitivity_var_hist)

In [ ]:
# Fig 3: Variance of Feedback Changes in 4 experiments with Single Colorbars

from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib import cm
#figcheck 
fig = plt.figure(figsize=(31,14))
ax1 = plt.subplot2grid((4,4), (0,0), projection=ccrs.PlateCarree(central_longitude=180.0))
ax2 = plt.subplot2grid((4,4), (0,1), projection=ccrs.PlateCarree(central_longitude=180.0))
ax3 = plt.subplot2grid((4,4), (0,2), projection=ccrs.PlateCarree(central_longitude=180.0))
ax4 = plt.subplot2grid((4,4), (0,3), projection=ccrs.PlateCarree(central_longitude=180.0))
ax5 = plt.subplot2grid((4,4), (1,0), projection=ccrs.PlateCarree(central_longitude=180.0))
ax6 = plt.subplot2grid((4,4), (1,1), projection=ccrs.PlateCarree(central_longitude=180.0))
ax7 = plt.subplot2grid((4,4), (1,2), projection=ccrs.PlateCarree(central_longitude=180.0))
ax8 = plt.subplot2grid((4,4), (1,3), projection=ccrs.PlateCarree(central_longitude=180.0))
ax9 = plt.subplot2grid((4,4), (2,0), projection=ccrs.PlateCarree(central_longitude=180.0))
ax10 = plt.subplot2grid((4,4), (2,1), projection=ccrs.PlateCarree(central_longitude=180.0))
ax11 = plt.subplot2grid((4,4), (2,2), projection=ccrs.PlateCarree(central_longitude=180.0))
ax12 = plt.subplot2grid((4,4), (2,3), projection=ccrs.PlateCarree(central_longitude=180.0))
ax13 = plt.subplot2grid((4,4), (3,0), projection=ccrs.PlateCarree(central_longitude=180.0))
ax14 = plt.subplot2grid((4,4), (3,1), projection=ccrs.PlateCarree(central_longitude=180.0))
ax15 = plt.subplot2grid((4,4), (3,2), projection=ccrs.PlateCarree(central_longitude=180.0))
ax16 = plt.subplot2grid((4,4), (3,3), projection=ccrs.PlateCarree(central_longitude=180.0))

gl1 = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl1.xlabels_top = False
gl2 = ax2.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl2.xlabels_top = False
gl3 = ax3.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl3.xlabels_top = False
gl4 = ax4.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl4.xlabels_top = False
gl5 = ax5.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl5.xlabels_top = False
gl6 = ax6.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl6.xlabels_top = False
gl7 = ax7.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl7.xlabels_top = False
gl8 = ax8.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl8.xlabels_top = False
gl9 = ax9.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl9.xlabels_top = False
gl10 = ax10.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl10.xlabels_top = False
gl11 = ax11.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl11.xlabels_top = False
gl12 = ax12.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl12.xlabels_top = False
gl13 = ax13.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl13.xlabels_top = False
gl14 = ax14.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl14.xlabels_top = False
gl15 = ax15.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl15.xlabels_top = False
gl16 = ax16.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl16.xlabels_top = False
ax1.coastlines(), ax2.coastlines(), ax3.coastlines(),ax4.coastlines(), ax5.coastlines(), ax6.coastlines(),ax7.coastlines(), ax8.coastlines(), ax9.coastlines(),ax10.coastlines(), ax11.coastlines(), ax12.coastlines(), ax13.coastlines(), ax14.coastlines(), ax15.coastlines(), ax16.coastlines()

ax1.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='k')
ax2.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='k')
ax3.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='k')
ax4.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='k')
ax5.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='k')
ax6.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='k')
ax7.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='k')
ax8.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='k')
ax9.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='k')
ax10.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='k')
ax11.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='k')
ax12.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='k')
ax13.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='k')
ax14.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='k')
ax15.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='k')
ax16.add_feature(cartopy.feature.LAND, zorder=100, edgecolor='k')

im1= dR_hat_var_amip.plot(ax=ax1,transform=ccrs.PlateCarree(),cmap='plasma',add_colorbar=False,vmax=8,vmin=0)
dR_sensitivity_var_amip.plot(ax=ax2,transform=ccrs.PlateCarree(),cmap='plasma',add_colorbar=False,vmin=0,vmax=8,)
dR_intrin_var_amip.plot(ax=ax3,transform=ccrs.PlateCarree(), cmap='plasma',add_colorbar=False,vmin=0,vmax=8,)
im2 = var_diff_amip.plot(ax=ax4,transform=ccrs.PlateCarree(), cmap='RdBu_r',add_colorbar=False,vmax=8,vmin=-8)
dR_hat_var_hist.plot(ax=ax5,transform=ccrs.PlateCarree(),cmap='plasma',add_colorbar=False, vmax=8,vmin=0)
dR_sensitivity_var_hist.plot(ax=ax6,transform=ccrs.PlateCarree(),cmap='plasma',add_colorbar=False, vmax=8,vmin=0)
dR_intrin_var_hist.plot(ax=ax7,transform=ccrs.PlateCarree(), cmap='plasma',add_colorbar=False, vmax=8,vmin=0)
var_diff_hist.plot(ax=ax8,transform=ccrs.PlateCarree(), cmap='RdBu_r',add_colorbar=False,vmax=8,vmin=-8,)
dR_hat_var_fast.plot(ax=ax9,transform=ccrs.PlateCarree(),cmap='plasma',add_colorbar=False,vmax=8,vmin=0)
dR_sensitivity_var_fast.plot(ax=ax10,transform=ccrs.PlateCarree(),cmap='plasma',add_colorbar=False,vmax=8,vmin=0)
dR_intrin_var_fast.plot(ax=ax11,transform=ccrs.PlateCarree(), cmap='plasma',add_colorbar=False,vmax=8,vmin=0)
var_diff_fast.plot(ax=ax12,transform=ccrs.PlateCarree(), cmap='RdBu_r', vmax=8,vmin=-8,add_colorbar=False)
dR_hat_var_slow.plot(ax=ax13,transform=ccrs.PlateCarree(),cmap='plasma',add_colorbar=False,vmax=8,vmin=0)
dR_sensitivity_var_slow.plot(ax=ax14,transform=ccrs.PlateCarree(),cmap='plasma',add_colorbar=False,vmax=8,vmin=0)
dR_intrin_var_slow.plot(ax=ax15,transform=ccrs.PlateCarree(), cmap='plasma',add_colorbar=False,vmax=8,vmin=0)
var_diff_slow.plot(ax=ax16,transform=ccrs.PlateCarree(), cmap='RdBu_r', vmax=8,vmin=-8,add_colorbar=False)


cbar_ax_var = fig.add_axes([0.695, 0.122, 0.008, 0.76])
cbar1=fig.colorbar(im1, cax=cbar_ax_var, ticks=np.arange(0,10,2),extend='max')
cbar1.set_label('($Wm^{-2}/K$)',size=18)
cbar1.ax.tick_params(labelsize=18);
cbar_ax_diff = fig.add_axes([0.898, 0.12, 0.008, 0.76])
cbar2=fig.colorbar(im2, cax=cbar_ax_diff, ticks=np.arange(-8,10,2),extend='both')
cbar2.set_label('($Wm^{-2}/K$)',size=18)
cbar2.ax.tick_params(labelsize=18);


make_textbox(ax1,'a')
make_textbox(ax2,'b')
make_textbox(ax3,'c')
make_textbox(ax4,'d')
make_textbox(ax5,'e')
make_textbox(ax6,'f')
make_textbox(ax7,'g')
make_textbox(ax8,'h')
make_textbox(ax9,'i')
make_textbox(ax10,'j')
make_textbox(ax11,'k')
make_textbox(ax12,'l')
make_textbox(ax13,'m')
make_textbox(ax14,'n')
make_textbox(ax15,'o')
make_textbox(ax16,'p')

plt.text(0.1, 0.779, 'AMIP', fontsize=24, transform=plt.gcf().transFigure,weight='bold',rotation='vertical')
plt.text(0.1, 0.546, 'Historical', fontsize=24, transform=plt.gcf().transFigure,fontweight='bold',rotation='vertical')
plt.text(0.1, 0.349, '4x$CO_{2}$-fast', fontsize=24, transform=plt.gcf().transFigure,fontweight='bold',rotation='vertical')
plt.text(0.1, 0.163, '4x$CO_{2}$-slow', fontsize=24, transform=plt.gcf().transFigure,fontweight='bold',rotation='vertical')

ax1.set_title('Var(dR)',fontsize=24)
ax2.set_title("Var($(dR/dX)'*\overline{dX}$)",fontsize=24)
ax3.set_title("Var($\overline{dR/dX}$*dX')",fontsize=24)
ax4.set_title("a - (b+c)",fontsize=24)


plt.savefig('/data/keeling/a/rytam2/ccf_model_spread/notebooks/figures/comp/Var_dR_4x4_16map_allmodels_singcolorbar_unifiedlimits.png',  dpi=400, bbox_inches='tight')